#Setup

In [1]:
!pip install pkbar

In [2]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
results_dir = base_dir + 'CNN/results_fashion/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 


Mounted at /content/gdrive/


In [3]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import time
import tqdm
import pkbar
import math

from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary


# Functions

In [4]:
def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
  fig.show()
  fig.write_html(results_dir + title + ".html")

def plot_acc_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis=dict(range=[0.75, 1]))
  fig.show()
  fig.write_html(results_dir + title + ".html")



In [5]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 64
test_BATCH_SIZE = 1000
EPOCHS = 20
GAP = 100

# Data Download

In [6]:
train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data', train=True, download=True,           
                       transform=transforms.Compose([                       
                               transforms.ToTensor()])),    
                        batch_size=BATCH_SIZE, shuffle=True, num_workers = 4, pin_memory = True)
test_loader = torch.utils.data.DataLoader(    
        datasets.FashionMNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()])),    
                        batch_size=test_BATCH_SIZE, shuffle=True, num_workers=4, pin_memory = True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [7]:
n_batches = np.ceil(len(train_loader.dataset)/BATCH_SIZE)
n_test_batches = np.ceil(len(test_loader.dataset)/test_BATCH_SIZE)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
torch.cuda.device_count()
torch.cuda.get_device_name(0)

Running on the GPU


'Tesla T4'

# Model

In [9]:
"""input image size for the original LeNet5 is 32x32, here is 28x28"""
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
def initialize_weights():
    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(1*5*5+1,  6)).clone().detach().requires_grad_(True)).to(device)
    W2 = torch.nn.init.xavier_uniform_((0.1*torch.randn(6*5*5+1,  16)).clone().detach().requires_grad_(True)).to(device)
    W3 = torch.nn.init.xavier_uniform_((0.1*torch.randn(16*4*4+1, 120)).clone().detach().requires_grad_(True)).to(device)
    W4 = torch.nn.init.xavier_uniform_((0.1*torch.randn(120+1,    84)).clone().detach().requires_grad_(True)).to(device)
    W5 = torch.nn.init.xavier_uniform_((0.1*torch.randn(84+1,     10)).clone().detach().requires_grad_(True)).to(device)
    Ws = [W1, W2, W3, W4, W5]
    return Ws

def LeNet5(x, return_all = False): 
    W1, W2, W3, W4, W5 = Ws
    x1 = F.relu(F.conv2d(x, W1[:-1].view(6,1,5,5), bias=W1[-1]))
    x2 = F.max_pool2d(x1, 2)
    x3 = F.relu(F.conv2d(x2, W2[:-1].view(16,6,5,5), bias=W2[-1]))
    x4 = F.max_pool2d(x3, 2)
    x5 = F.relu(x4.view(-1, 16*4*4).mm(W3[:-1]) + W3[-1])
    x6 = F.relu(x5.mm(W4[:-1]) + W4[-1])
    # x = F.dropout(x, 0.3, training = True)
    y = x6.mm(W5[:-1]) + W5[-1]
    if return_all:
      return F.log_softmax(y, dim = 1), [x, x1, x2, x3, x4, x5, x6, y]
    return F.log_softmax(y, dim=1)


# Loss Function

In [10]:
def train_loss(data, target):
    y = LeNet5(data)
    loss = F.nll_loss(y, target)
    # loss = F.cross_entropy(y, target)
    _, max_indices = torch.max(y, dim = 1)
    accuracy = (max_indices == target).sum(dtype=torch.float32)/max_indices.size(0)
    return loss, accuracy

def test_loss():
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            y = LeNet5(data)
            loss += F.nll_loss(y, target)
            _, pred = torch.max(y, dim=1)
            accuracy += (pred == target).sum(dtype=torch.float32)/pred.size(0)
    return loss.item()/n_test_batches, accuracy.item()/n_test_batches

def save_start_condition(trainlosslist, testlosslist,trainacclist, testacclist, timelist):
    trainloss = 0.0
    trainacc = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      loss, accuracy = train_loss(data, target)
      trainloss += loss
      trainacc += accuracy
      
  
    timelist.append(0)

    testloss, testacc = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    trainacclist.append(trainacc.item()/n_batches)
    testlosslist.append(testloss)
    testacclist.append(testacc)
    print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    .format(0, trainlosslist[-1], testlosslist[-1], trainacclist[-1], testacclist[-1],np.sum(timelist)))

# ALL TEST

## SGD

In [11]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy
        
        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*grads[i]

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)

    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])

    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'sgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 7s 8ms/step - loss: 0.5971 - acc: 0.7780 - val_loss: 0.5506 - val_acc: 0.7897
Epoch: 2/20
938/938 [==============================] - 7s 8ms/step - loss: 0.3921 - acc: 0.8547 - val_loss: 0.3851 - val_acc: 0.8612
Epoch: 3/20
938/938 [==============================] - 7s 8ms/step - loss: 0.3452 - acc: 0.8726 - val_loss: 0.3752 - val_acc: 0.8603
Epoch: 4/20
938/938 [==============================] - 7s 8ms/step - loss: 0.3220 - acc: 0.8798 - val_loss: 0.3578 - val_acc: 0.8669
Epoch: 5/20
938/938 [==============================] - 7s 8ms/step - loss: 0.3072 - acc: 0.8854 - val_loss: 0.3573 - val_acc: 0.8691
Epoch: 6/20
938/938 [==============================] - 7s 8ms/step - loss: 0.2980 - acc: 0.8897 - val_loss: 0.3378 - val_acc: 0.8751
Epoch: 7/20
938/938 [=============================

## Adam

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()

m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
step_size = 0.005
cnt = 0
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws)#, create_graph=True)
        trainloss += loss
        trainacc += accuracy
        

        with torch.no_grad():
            lmbd = min(cnt/(cnt+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(cnt/(cnt+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for i in range(len(Ws)):
                Ws[i] -= step_size*(m0[i]/torch.sqrt(v0[i] + 1e-8))
            cnt = cnt + 1
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1    
        
    t1 = time.time() - t0
    times.append(t1)

    testloss, testacc = test_loss()

    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    # .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 7s 8ms/step - loss: 0.4894 - acc: 0.8187 - val_loss: 0.3975 - val_acc: 0.8534
Epoch: 2/20
938/938 [==============================] - 7s 8ms/step - loss: 0.3286 - acc: 0.8771 - val_loss: 0.3450 - val_acc: 0.8756
Epoch: 3/20
938/938 [==============================] - 7s 8ms/step - loss: 0.2793 - acc: 0.8960 - val_loss: 0.3220 - val_acc: 0.8859
Epoch: 4/20
938/938 [==============================] - 8s 8ms/step - loss: 0.2475 - acc: 0.9073 - val_loss: 0.3094 - val_acc: 0.8860
Epoch: 5/20
938/938 [==============================] - 7s 8ms/step - loss: 0.2262 - acc: 0.9146 - val_loss: 0.3048 - val_acc: 0.8903
Epoch: 6/20
938/938 [==============================] - 8s 8ms/step - loss: 0.2106 - acc: 0.9210 - val_loss: 0.2934 - val_acc: 0.8943
Epoch: 7/20
938/938 [=============================

## Full Kronecker

In [13]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

938/938 [==============================] - 17s 18ms/step - loss: 0.4761 - acc: 0.8255 - val_loss: 0.4210 - val_acc: 0.8399
Epoch: 2/20
938/938 [==============================] - 17s 18ms/step - loss: 0.2988 - acc: 0.8892 - val_loss: 0.3366 - val_acc: 0.8789
Epoch: 3/20
938/938 [==============================] - 17s 18ms/step - loss: 0.2313 - acc: 0.9136 - val_loss: 0.3072 - val_acc: 0.8913
Epoch: 4/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1859 - acc: 0.9308 - val_loss: 0.3041 - val_acc: 0.8979
Epoch: 5/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1564 - acc: 0.9427 - val_loss: 0.3005 - val_acc: 0.9020
Epoch: 6/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1389 - acc: 0.9496 - val_loss: 0.3074 - val_acc: 0.9018
Epoch: 7/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1284 - acc: 0.9538 - val_loss: 0.3106 - val_acc: 0.9015
Epoch: 8/20
938/938 [==============================] - 17s 18ms/ste

## DPSGD Approach 1

In [14]:
_tiny = 1.2e-38 
 # pi = (torch.trace(Ql)*Qr.shape[0])/(torch.trace(Qr)*Ql.shape[0])
    # 

    
def precond_grad_kron(Ql, Qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd))*IR)

    return P1.mm(Grad).mm(P2)

def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    # print(r, M, lambd)
    if r > 3/4:
      lambd = lambd*omega
    elif r < 1/4:
      lambd = lambd / omega
    return lambd


In [15]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after

eta = 1e-5
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
                #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
                loss2 = F.nll_loss(LeNet5(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)

    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 18s 19ms/step - loss: 0.4835 - acc: 0.8203 - val_loss: 0.4164 - val_acc: 0.8450
Epoch: 2/20
938/938 [==============================] - 18s 20ms/step - loss: 0.2795 - acc: 0.8941 - val_loss: 0.3237 - val_acc: 0.8801
Epoch: 3/20
938/938 [==============================] - 18s 19ms/step - loss: 0.2136 - acc: 0.9191 - val_loss: 0.2892 - val_acc: 0.8930
Epoch: 4/20
938/938 [==============================] - 18s 19ms/step - loss: 0.1676 - acc: 0.9369 - val_loss: 0.2983 - val_acc: 0.8941
Epoch: 5/20
938/938 [==============================] - 18s 19ms/step - loss: 0.1385 - acc: 0.9492 - val_loss: 0.2949 - val_acc: 0.9006
Epoch: 6/20
938/938 [==============================] - 18s 19ms/step - loss: 0.1210 - acc: 0.9560 - val_loss: 0.3004 - val_acc: 0.9017
Epoch: 7/20
938/938 [=================

In [16]:
full_data, full_target = [],[]
for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data, target
      full_data.extend(data)
      full_target.extend(data)
full_data = torch.stack(full_data).to(device)
full_target = torch.stack(full_target)#.to(device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [ ]:
# torch.manual_seed(1)
# Ws = initialize_weights()
# Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
# dQs = [[torch.ones(W.shape[0],1).to(device), torch.ones(1,W.shape[1]).to(device)] for W in Ws]

# step_size = 0.1
# grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
# TrainLoss, TestLoss = [], []
# TrainAcc, TestAcc = [], []
# times = []
# save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)
# full_pre_grads = [torch.zeros(W.shape).to(device) for W in Ws]
# lambd = 1
# update_after = 5
# omega = (19/20)**update_after

# eta = 1e-5
# for epoch in range(EPOCHS):
#     kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
#     trainloss = 0.0
#     trainacc = 0.0
#     n = 0
#     t0 = time.time()
#     loss, accuracy = train_loss(full_data, fulltarget)
#     grads = grad(trainloss, Ws, create_graph=True)
#     v = [torch.randn(W.shape).to(device) for W in Ws]
#     Hv = grad(grads, Ws, v)
#     with torch.no_grad():
#         Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
#         full_pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
      

    
#     for batch_idx, (data, target) in enumerate(train_loader):
      
#         data, target = data.to(device), target.to(device)
#         loss, accuracy = train_loss(data, target)
        
#         grads = grad(loss, Ws, create_graph=True)
        
#         trainloss += loss
#         trainacc += accuracy          
        
#         with torch.no_grad():
#             Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
#             pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
#             damp_grads = [((lambd+eta)**0.5)*g for g in grads]
#             pre_grads = [pg+dg for (pg, dg) in zip(pre_grads, damp_grads)] 
#             grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
#             step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

#             for i in range(len(Ws)):
#                 Ws[i] -= step_adjust*step_size*(pre_grads[i] - full_pre_grads[i])

#             if n % update_after == 0 and lambd > 1e-10:
#                 M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
#                 # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
#                 # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
#                 #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
#                 loss2 = F.nll_loss(LeNet5(data), target)
#                 loss1 = loss
#                 lambd = update_lambda(loss1, loss2, M,  lambd, omega)

#         kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
#         n += 1

#     t1 = time.time() - t0
#     times.append(t1)
#     TrainLoss.append(trainloss.item()/n_batches)
#     TrainAcc.append(trainacc.item()/n_batches)

#     testloss, testacc = test_loss()

#     TestLoss.append(testloss)
#     TestAcc.append(testacc)
#     kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
#     step_size = 0.01**(1/9)*step_size
#     # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
#     #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

# # scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

## DPSGD Approach 2

In [18]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
dQs = [[torch.ones(W.shape[0],1).to(device), torch.ones(1,W.shape[1]).to(device)] for W in Ws]

step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after

eta = 1e-5
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            damp_grads = [((lambd+eta)**0.5)*g for g in grads]
            pre_grads = [pg+dg for (pg, dg) in zip(pre_grads, damp_grads)] 
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
                #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
                loss2 = F.nll_loss(LeNet5(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)

    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron_damped2.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 17s 18ms/step - loss: 0.4691 - acc: 0.8291 - val_loss: 0.3893 - val_acc: 0.8508
Epoch: 2/20
938/938 [==============================] - 17s 18ms/step - loss: 0.2822 - acc: 0.8955 - val_loss: 0.3207 - val_acc: 0.8844
Epoch: 3/20
938/938 [==============================] - 17s 18ms/step - loss: 0.2137 - acc: 0.9204 - val_loss: 0.2894 - val_acc: 0.8955
Epoch: 4/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1665 - acc: 0.9382 - val_loss: 0.2848 - val_acc: 0.9019
Epoch: 5/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1367 - acc: 0.9498 - val_loss: 0.2906 - val_acc: 0.9056
Epoch: 6/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1195 - acc: 0.9571 - val_loss: 0.2969 - val_acc: 0.9063
Epoch: 7/20
938/938 [=================

## D-PDSGD w/ momentum

In [24]:
def precond_kron(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd))*IR)

    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[torch.zeros(W.shape[0]).to(device), torch.zeros(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5
beta = 0.7

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    n = 0
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
   
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            beta = min(n/(n+1), 0.7)
            Ps = [precond_kron(q[0], q[1], p[0], p[1], beta) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.nll_loss(LeNet5(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M, lambd, omega)
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)

    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'mod_psgd1.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 19s 20ms/step - loss: 0.4690 - acc: 0.8265 - val_loss: 0.4447 - val_acc: 0.8379
Epoch: 2/20
938/938 [==============================] - 19s 20ms/step - loss: 0.2772 - acc: 0.8948 - val_loss: 0.3203 - val_acc: 0.8803
Epoch: 3/20
938/938 [==============================] - 19s 20ms/step - loss: 0.2097 - acc: 0.9212 - val_loss: 0.2901 - val_acc: 0.8925
Epoch: 4/20
938/938 [==============================] - 19s 20ms/step - loss: 0.1631 - acc: 0.9391 - val_loss: 0.2892 - val_acc: 0.8998
Epoch: 5/20
938/938 [==============================] - 19s 20ms/step - loss: 0.1332 - acc: 0.9507 - val_loss: 0.3007 - val_acc: 0.9000
Epoch: 6/20
938/938 [==============================] - 19s 20ms/step - loss: 0.1154 - acc: 0.9580 - val_loss: 0.3075 - val_acc: 0.9005
Epoch: 7/20
938/938 [=================

##KFAC

In [26]:
torch.manual_seed(1)
Ws = initialize_weights()
from kfac import KFAC
import torch.nn as nn
import torch.optim as optim

class LeNet5_K(nn.Module):
    def __init__(self):
        super(LeNet5_K, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

def test_loss_K(model):
    model.eval()
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            y = model(data)
            loss += F.nll_loss(y, target)
            _, pred = torch.max(y, dim=1)
            accuracy += (pred == target).sum(dtype=torch.float32)/pred.size(0)
    return loss.item()/n_test_batches, accuracy/n_test_batches

model = LeNet5_K().to(device)
preconditioner = KFAC(model, 0.001, alpha=0.05, sua = True)
lr0 = 0.01
optimizer = optim.SGD(model.parameters(), lr=lr0)
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    model.train()
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        _, max_ind = torch.max(output, dim = 1)
        accuracy = (max_ind == target).sum(dtype=torch.float32)/max_ind.size(0) 

        trainloss += loss
        trainacc += accuracy

        loss.backward()
        preconditioner.step()
        optimizer.step()

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1
        
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    
    lr0 = 0.01**(1/9)*lr0
    optimizer.param_groups[0]['lr'] = lr0
    testloss, testacc = test_loss_K(model)

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'KFAC.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4305425127432034; test loss: 2.429814910888672, train_accuracy: 0.10026319296375266, test_accuracy:0.10040000677108765, time: 0
Epoch: 1/20
  0/938 [..............................] - ETA: 0s - loss: 0.0000e+00 - acc: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.



938/938 [==============================] - 12s 13ms/step - loss: 0.4162 - acc: 0.8475 - val_loss: 0.3363 - val_acc: 0.8751
Epoch: 2/20
938/938 [==============================] - 12s 13ms/step - loss: 0.2718 - acc: 0.9002 - val_loss: 0.3077 - val_acc: 0.8842
Epoch: 3/20
938/938 [==============================] - 13s 14ms/step - loss: 0.2204 - acc: 0.9175 - val_loss: 0.2923 - val_acc: 0.8936
Epoch: 4/20
938/938 [==============================] - 12s 13ms/step - loss: 0.1865 - acc: 0.9321 - val_loss: 0.2926 - val_acc: 0.8971
Epoch: 5/20
938/938 [==============================] - 12s 13ms/step - loss: 0.1648 - acc: 0.9401 - val_loss: 0.2938 - val_acc: 0.8959
Epoch: 6/20
938/938 [==============================] - 12s 13ms/step - loss: 0.1510 - acc: 0.9462 - val_loss: 0.2967 - val_acc: 0.8976
Epoch: 7/20
938/938 [==============================] - 13s 13ms/step - loss: 0.1420 - acc: 0.9502 - val_loss: 0.2994 - val_acc: 0.8991
Epoch: 8/20
938/938 [==============================] - 13s 13ms/ste

## Shampoo

In [27]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[0.01*torch.eye(W.shape[0]).to(device), 0.01*torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.5
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

def matrix_power(matrix, power):
    # use CPU for svd for speed up
    matrix = matrix.cpu()
    u, s, v = torch.svd(matrix)
    return (u @ s.pow_(power).diag() @ v.t()).cuda()

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy
        
        with torch.no_grad():
            Qs = [[q[0] + g.mm(g.t()), q[1] + (g.t()).mm(g)] for (q, g) in zip(Qs, grads)]
            inv_Qs = [[matrix_power(q[0], -1/4), matrix_power(q[1], -1/4)]for q in Qs]
            pre_grads = [q[0].mm(g).mm(q[1]) for (q, g) in zip(inv_Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.1**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'shampoo.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 2.4306401169376333; test loss: 2.4298145294189455, train_accuracy: 0.10022987739872068, test_accuracy:0.1004000186920166, time: 0
Epoch: 1/20
938/938 [==============================] - 22s 24ms/step - loss: 0.4525 - acc: 0.8305 - val_loss: 0.3641 - val_acc: 0.8678
Epoch: 2/20
938/938 [==============================] - 22s 24ms/step - loss: 0.2987 - acc: 0.8873 - val_loss: 0.3181 - val_acc: 0.8825
Epoch: 3/20
938/938 [==============================] - 22s 24ms/step - loss: 0.2575 - acc: 0.9029 - val_loss: 0.3088 - val_acc: 0.8873
Epoch: 4/20
938/938 [==============================] - 22s 24ms/step - loss: 0.2348 - acc: 0.9122 - val_loss: 0.3047 - val_acc: 0.8888
Epoch: 5/20
938/938 [==============================] - 22s 24ms/step - loss: 0.2183 - acc: 0.9191 - val_loss: 0.2977 - val_acc: 0.8926
Epoch: 6/20
938/938 [==============================] - 22s 24ms/step - loss: 0.2063 - acc: 0.9234 - val_loss: 0.2919 - val_acc: 0.8955
Epoch: 7/20
938/938 [=================

# Comparison

In [ ]:
opts = ['sgd','adam','KFAC', 'shampoo','Kron','Kron_damped', 'SAMDPSGD']

total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}
train_err={}
test_err = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#000000','#33E0FF', '#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
  # print(opt)
  data = opts_data[opt]
  times[opt] = data.get('Time')
  train_times[opt] = np.cumsum(times[opt])
  test_times[opt] = np.cumsum(times[opt])
  total_train_time[opt] = np.sum(times[opt])
  train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
  train_accs[opt] = data.get('TrainAccuracy').reshape(EPOCHS+1,)
  test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)
  test_accs[opt] = data.get('TestAccuracy').reshape(EPOCHS+1)
  train_err[opt] = (1-data.get('TrainAccuracy')).reshape(EPOCHS+1,)
  

for opt in ['kron_damped','SAMDPSGD']:
  test_err[opt] = (1-opts_data[opt].get('TestAccuracy')).reshape(EPOCHS+1,)

In [ ]:

fig = go.Figure()
i = 0
for opt in ['kron', 'kron_damped','SAMDPSGD']:
  fig.add_trace(go.Scatter(y=test_err[opt], name = opt, mode='lines', line = dict(color = colors[i])))
  i = i + 1

fig.update_layout(title='test error', xaxis_title='epochs', yaxis_title='test error', yaxis_type="log")
fig.show()

In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')

In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_err,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
# plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_err,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
# plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')

In [ ]:
# plot train_losses vs Iterations
plot_acc_metrics(None,train_accs,'Train Accuracy vs EPOCHS', 'EPOCHS','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(None,test_accs,'Test Accuracy vs EPOCHS', 'EPOCHS','Test Accuracys')
# # plot test_losses vs Iterations
plot_acc_metrics(train_times,train_accs,'Train Accuracy vs Time', 'Time','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(test_times,test_accs,'Test Accuracy vs Time', 'Time','Test Accuracy')